# Procedure to make data for this project

# Rideshare Trip Data

First, manually download a csv from the City of Chicago website
```
https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips/m6dm-c72p
```
Click on the export box then click on the csv box. Notice the file is large, about 34 GB.

## Setup

In [2]:
import pandas as pd
import psycopg2 as pg
import os
import sys
import pandas.io.sql as pd_sql
import numpy as np
import sqlalchemy
project_dir = str(os.path.dirname(os.path.abspath('')))
print(project_dir)
sys.path.append(project_dir)

/Users/erik/metis/data_hailing


I ran this in the command line:
```
psql
CREATE DATABASE data_hailing;
```

Convert the csv to a pandas-friendly format that sql will recognize, given the table schema that pandas will be generating

## Create SQL friendly csv, using pandas

In [ ]:
=chunksize = 10 ** 6
in_file_path = project_dir + '/data/raw/' + 'Transportation_Network_Providers_-_Trips.csv'
first = True

out_file_path = project_dir + '/data/interim/' + 'tnp_trips_2019_pandas.csv'
assert not os.path.exists(out_file_path), 'file already exists'

for i, chunk in enumerate(pd.read_csv(in_file_path, chunksize=chunksize)):
    
    chunk.columns = [s.lower().replace(" ", "_") for s in chunk.columns]
    
    chunk = chunk.drop(['trip_id', 'pickup_centroid_location', 'dropoff_centroid_location'], axis=1)
    chunk['trip_start_timestamp'] = pd.to_datetime(chunk['trip_start_timestamp'], format="%m/%d/%Y %I:%M:%S %p")
    chunk['trip_end_timestamp'] = pd.to_datetime(chunk['trip_end_timestamp'], format="%m/%d/%Y %I:%M:%S %p")
    
    chunk = chunk[chunk['trip_start_timestamp'].dt.year == 2019]
    

    if first:
#         df = pd.DataFrame(chunk)
        first = False
        chunk.to_csv(out_file_path, index=False)
    else:
#         df = df.append(chunk)
        chunk.to_csv(out_file_path, index=False, mode='a', header=False)
    print(i)



## copy csv into SQL data base

I ran this in psql:

``` sql
data_hailing=# drop table tnp_trips_2019_pandas


data_hailing=# create table tnp_trips_2019_pandas
(
    trip_start_timestamp timestamp,
    trip_end_timestamp timestamp,
    trip_seconds NUMERIC,
    trip_miles NUMERIC,
    pickup_census_tract NUMERIC,
    dropoff_census_tract NUMERIC,
    pickup_community_area NUMERIC,
    dropoff_community_area NUMERIC,
    fare NUMERIC,
    tip NUMERIC,
    additional_charges NUMERIC,
    trip_total NUMERIC,
    shared_trip_authorized TEXT,
    trips_pooled NUMERIC,
    pickup_centroid_latitude NUMERIC,
    pickup_centroid_longitude NUMERIC,
    dropoff_centroid_latitude NUMERIC,
    dropoff_centroid_longitude NUMERIC
);


CREATE TABLE

```


And then I copied in the data from the csv I created above

```
data_hailing=# \copy tnp_trips_2019_pandas FROM '/U`sers/erik/metis/data_hailing/data/interim/tnp_trips_2019_pandas.csv' DELIMITER ',' CSV HEADER;
COPY 111259304
```
That's 111 million rows copied.

I also ran this

```
ALTER TABLE tnp_trips_2019_pandas ALTER COLUMN shared_trip_authorized type bool USING shared_trip_authorized::bool;
```
to convert the shared_trip_authorized column to a bool type.

### Table Schema

This is the resulting table schema:
```
data_hailing=# \d tnp_trips_2019_pandas
                           Table "public.tnp_trips_2019_pandas"
           Column           |            Type             | Collation | Nullable | Default
----------------------------+-----------------------------+-----------+----------+---------
 trip_start_timestamp       | timestamp without time zone |           |          |
 trip_end_timestamp         | timestamp without time zone |           |          |
 trip_seconds               | numeric                     |           |          |
 trip_miles                 | numeric                     |           |          |
 pickup_census_tract        | numeric                     |           |          |
 dropoff_census_tract       | numeric                     |           |          |
 pickup_community_area      | numeric                     |           |          |
 dropoff_community_area     | numeric                     |           |          |
 fare                       | numeric                     |           |          |
 tip                        | numeric                     |           |          |
 additional_charges         | numeric                     |           |          |
 trip_total                 | numeric                     |           |          |
 shared_trip_authorized     | boolean                     |           |          |
 trips_pooled               | numeric                     |           |          |
 pickup_centroid_latitude   | numeric                     |           |          |
 pickup_centroid_longitude  | numeric                     |           |          |
 dropoff_centroid_latitude  | numeric                     |           |          |
 dropoff_centroid_longitude | numeric                     |           |          |
 ```

## Load data from SQL into pandas (test example)

In [4]:
engine = sqlalchemy.create_engine('postgresql://localhost:5432/data_hailing')
query = "SELECT * FROM tnp_trips_2019_pandas WHERE DATE(trip_start_timestamp) = '2019-01-01'limit 500;"
df_test = pd.read_sql_query(query, engine)

In [5]:
df_test

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total,shared_trip_authorized,trips_pooled,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
0,2019-01-01 17:15:00,2019-01-01 17:15:00,778.0,3.5,NaN,NaN,26.0,25.0,10.0,0.0,0.00,10.00,True,1.0,41.878594,-87.730232,41.890609,-87.756047
1,2019-01-01 02:15:00,2019-01-01 02:30:00,868.0,1.6,1.703107e+10,1.703106e+10,7.0,6.0,5.0,0.0,2.50,7.50,True,2.0,41.921855,-87.646211,41.943155,-87.640698
2,2019-01-01 04:15:00,2019-01-01 05:00:00,1952.0,22.3,1.703132e+10,NaN,32.0,NaN,30.0,0.0,7.70,37.70,False,1.0,41.870607,-87.622173,NaN,NaN
3,2019-01-01 03:30:00,2019-01-01 03:45:00,962.0,5.5,NaN,NaN,12.0,NaN,10.0,2.0,2.50,14.50,False,1.0,41.993930,-87.758354,NaN,NaN
4,2019-01-01 20:30:00,2019-01-01 20:45:00,554.0,1.9,1.703128e+10,1.703108e+10,28.0,8.0,5.0,0.0,2.55,7.55,False,1.0,41.879255,-87.642649,41.890922,-87.618868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2019-01-01 17:15:00,2019-01-01 17:30:00,765.0,2.6,1.703184e+10,1.703106e+10,8.0,6.0,7.5,0.0,2.55,10.05,False,1.0,41.901567,-87.638404,41.934540,-87.643023
496,2019-01-01 10:15:00,2019-01-01 10:30:00,604.0,2.3,NaN,NaN,33.0,31.0,7.5,2.0,2.55,12.05,False,1.0,41.857184,-87.620335,41.850266,-87.667569
497,2019-01-01 09:15:00,2019-01-01 09:15:00,301.0,1.8,1.703107e+10,1.703108e+10,7.0,8.0,5.0,0.0,2.50,7.50,False,1.0,41.921778,-87.651062,41.907492,-87.635760
498,2019-01-01 02:30:00,2019-01-01 03:00:00,1094.0,4.0,1.703108e+10,1.703124e+10,8.0,24.0,20.0,0.0,2.50,22.50,False,1.0,41.890922,-87.618868,41.899422,-87.684490


In [4]:
df_test.dtypes

trip_start_timestamp          datetime64[ns]
trip_end_timestamp            datetime64[ns]
trip_seconds                         float64
trip_miles                           float64
pickup_census_tract                  float64
dropoff_census_tract                 float64
pickup_community_area                float64
dropoff_community_area               float64
fare                                 float64
tip                                  float64
additional_charges                   float64
trip_total                           float64
shared_trip_authorized                  bool
trips_pooled                         float64
pickup_centroid_latitude             float64
pickup_centroid_longitude            float64
dropoff_centroid_latitude            float64
dropoff_centroid_longitude           float64
dtype: object

# Income Data for City of Chicago

Download the file here: https://data.cityofchicago.org/Health-Human-Services/Per-Capita-Income/r6ad-wvtk
Download it as a csv and place it in
`project dir, /data/raw, Per_Capita_Income_Chicago.csv`


In [3]:
# clean up the column names
per_capita = pd.read_csv(project_dir + '/data/raw/' + 
                         'per_capita_income_chicago.csv')

In [8]:
per_capita.columns = [s.lower().replace(" ", "_") for
                     s in per_capita.columns]
out_file_path = (project_dir + '/data/interim/' + 
                 'per_capita_income_chicago_tidy.csv')
# assert not os.path.exists(out_file_path), 'file already exists'
# per_capita.to_csv(out_file_path, index=False)

Getting these data in SQL

I did run the following commands:
```
create table per_capita_income_chicago_tidy
(
	community_area_number NUMERIC,
	community_area_name TEXT,
	percent_of_housing_crowded NUMERIC,
	percent_households_below_poverty NUMERIC,
	"percent_aged_16+_unemployed" NUMERIC,
	"percent_aged_25+_without_high_school_diploma" NUMERIC,
	percent_aged_under_18_or_over_64 NUMERIC,
	per_capita_income_ INT,
	hardship_index NUMERIC
);

result:
CREATE TABLE



\copy per_capita_income_chicago_tidy FROM '/Users/erik/metis/data_hailing/data/interim/per_capita_income_chicago_tidy.csv' DELIMITER ',' CSV HEADER;

result:
COPY 78

\d per_capita_income_chicago_tidy

result:

                      Table "public.per_capita_income_chicago_tidy"
                    Column                    |  Type   | Collation | Nullable | Default
----------------------------------------------+---------+-----------+----------+---------
 community_area_number                        | numeric |           |          |
 community_area_name                          | text    |           |          |
 percent_of_housing_crowded                   | numeric |           |          |
 percent_households_below_poverty             | numeric |           |          |
 percent_aged_16+_unemployed                  | numeric |           |          |
 percent_aged_25+_without_high_school_diploma | numeric |           |          |
 percent_aged_under_18_or_over_64             | numeric |           |          |
 per_capita_income_                           | integer |           |          |
 hardship_index                               | numeric |           |          |


select community_area_number, community_area_name, per_capita_income_ from per_capita_income_chicago_tidy limit 10;

result:
 community_area_number | community_area_name | per_capita_income_
-----------------------+---------------------+--------------------
                   1.0 | Rogers Park         |              23939
                   2.0 | West Ridge          |              23040
                   3.0 | Uptown              |              35787
                   4.0 | Lincoln Square      |              37524
                   5.0 | North Center        |              57123
                   6.0 | Lake View           |              60058
                   7.0 | Lincoln Park        |              71551
                   8.0 | Near North Side     |              88669
                   9.0 | Edison Park         |              40959
                  10.0 | Norwood Park        |              32875


```


# Community Areas map for city of chicago

I did:
Download the file from 
https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-current-/cauq-8yn6
and save it as a csv to:
`project dir, /data/raw, Per_Capita_Income_Chicago.csv
chicago_community_areas_map.geojson`

(note the original file name was `'Boundaries - Community Areas (current).geojson'`)